In [4]:
import numpy as np

The seed laser passes through a crystal of length L ($L_c$=1cm). In the case that the crystal isn't pumped first,
it arrives at a waist at 50 cm from the end of the crystal. Now, assume it is pumped and heated, giving rise to
a thermal lensing effect. We want to find the ABCD matrix that gets us between these two cases.
We need to go backwards through the unheated crystal and then forward through the heated crystal.
The matrix to pass through the crystal with $n_2 = 0$ is 
$$M_c = \pmatrix{1 & \frac{L_c}{n_0} \cr 0 & 1} $$
The crystal matrix with n_0 and n_2 is

$$\bar M_c = \pmatrix{\cos(\gamma L) & \frac{1}{n_0 \gamma} \sin(\gamma L)\cr 
            -(n_0 \gamma) \sin(\gamma L) & \cos(\gamma L) } $$

This acts on a ray via
$$\pmatrix{ x_f \cr \theta_f } = \pmatrix{ A & B \cr C & D }\pmatrix{x_i \cr \theta_i}$$
Next, we consider the drift from the crystal to the wavefront sensor of length $L_d$ = 0.5m, the matrix of which is

$$M_d = \pmatrix{1 & L_d \cr 0 & 1} $$


The total matrix we want to apply to transform between the two measurmed wavefronts is
$$M_T = M_d \bar M_c M_c^{-1} M_d^{-1}$$

The index of refraction is given by $$n(r) = n_0 - \frac{1}{2} n_2 r^2$$ 
and $\gamma$ is defined as 
$$\gamma = \sqrt{\frac{n_2}{n_0}}$$

Lengths are given in m and angles in radians.

In [8]:
n0 = 1.76
#n2=1.6e-3 #cm^-2
n2 = 16 #m^-2

Lc = 0.01
Ld = 0.5
Matcinv = np.array( [[ 1, -Lc/n0], [0, 1]])
gamma = np.sqrt(n2/n0)
A = np.cos(gamma*Lc)
B = 1/(n0 * gamma) * np.sin(gamma*Lc)
C = -n0 * gamma * np.sin(gamma * Lc)
D = np.cos(gamma*Lc)
Matcbar = np.array( [[ A, B], [C, D]] )
Matd = np.array( [[ 1, Ld], [0, 1]] )
Matdinv = np.array( [[ 1, -Ld], [0, 1]] )
Mattot = np.matmul(np.matmul(np.matmul(Matd,Matcbar),Matcinv),Matdinv)
print(gamma)
print(Mattot)

3.0151134457776365
[[ 0.91955761  0.04045014]
 [-0.15997576  1.08044232]]


Now we construct the initial moment matrix $\Sigma_i$ and transform it via $M$, using the fact that
$$\Sigma_f = M\Sigma_i M^T$$
where $\Sigma_{i,f}$ are the initial and final second moment matrices respectively.

In [9]:
#sigmax = 64e-6 #rms beam size [m]
sigmax = 1e-3
lam = 799.89e-9 #wavelength [m]
sigtheta = lam/(4*np.pi*sigmax) #rms divergence [rad]

sigmai = np.array( [[sigmax**2, 0], [0, sigtheta**2] ] ) 
sigmaf = np.matmul(np.matmul(Mattot, sigmai),np.transpose(Mattot))
print(sigmai)
print(sigmaf)

[[1.00000000e-06 0.00000000e+00]
 [0.00000000e+00 4.05173289e-09]]
[[ 8.45592827e-07 -1.46929849e-07]
 [-1.46929849e-07  3.03220565e-08]]


Now, we relate this to the phase of a Gaussian wavefront.
We consider a coherent Gaussian given by
$$E(x) = E_0 e^{-\frac{a+i b}{2} x^2} $$
We define a second moment matrix via
$$\Sigma = \pmatrix{\Sigma_{xx} & \Sigma_{x\theta} \cr \Sigma_{x\theta} & \Sigma_{\theta\theta}} $$
We can relate the second moment matrix to the real parameters $a$ and $b$.
$$\Sigma_{xx} = \frac{1}{2a}$$
$$\Sigma_{x\theta} = \frac{\lambda}{4\pi} \frac{b}{a}$$
$$\Sigma_{\theta \theta} = \frac{1}{2} \left(\frac{\lambda}{2\pi}\right) \left( a+ \frac{b}{a}\right) $$

From the equations for $\Sigma_{xx}$ and $\Sigma_{x\theta}$, we find
$$ b = \frac{2\pi}{\lambda} \frac{\Sigma_{x\theta}}{\Sigma_{xx}} $$

In [10]:
lambda_micron = 0.8
b = (2*np.pi)/lam * sigmaf[0,1]/sigmaf[0,0]
b_micron = lambda_micron * b
print(b_micron)

-1091913.7387110577


In [11]:
alpha_micron_m2 = 6.5e-7 #This number from JVT email
alpha_meter_m2 = 1e12*alpha_micron_m2
print(alpha_meter_m2)

650000.0


In [13]:
-2*alpha_meter_m2/b

0.952456190566538

This factor of 2 may come from the 1D treatment given here. x-> r